<a href="https://colab.research.google.com/github/lalesafarzade/HUMOR_DETECTION_NLP/blob/main/tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
src = list(files.upload().values())[0]
open('mylib.py','wb').write(src)
import mylib

Saving utils.py to utils.py


In [5]:
uploaded = files.upload()

Saving dataset.csv to dataset.csv


In [118]:
import numpy as np
import warnings
from tensorflow.keras.callbacks import EarlyStopping
warnings.filterwarnings("ignore")
import tensorflow as tf
import tensorflow_hub as hub
from utils import data_splitter,text_process,build_freqs
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout
from utils import hub_model
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.8.0
Eager mode:  True
Hub version:  0.12.0
GPU is available


In [113]:
df = pd.read_csv('dataset.csv')
df['humor']=df['humor'].map({True:1,False:0})
df.head(2)

,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",0
1,Watch: darvish gave hitter whiplash with slow ...,0


In [114]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['humor'], test_size=0.3)
X_train=np.array(X_train)
y_train=np.array(y_train)
X_val=np.array(X_test)
y_val=np.array(y_test)
X_test=X_val[:1000]
y_test=y_val[:1000]
X_val=X_val[1000:]
y_val=y_val[1000:]

In [115]:
X_train[:5]

array(['How my 4-year-old helped me create something extraordinary',
       'The greatest gift a mom can give is self-love',
       'What do you do when you love a hotel? you mariott',
       'If the election were held today, trump would get 99 electoral college votes',
       "Bed quota fuels 'inhumane' and 'unnecessary' immigrant detention: report"],
      dtype=object)

In [116]:
y_train[:5]

array([0, 0, 1, 0, 0])

In [119]:

hub_model = hub_model()
hub_layer = hub.KerasLayer(hub_model, input_shape=[], dtype=tf.string, trainable=True)
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(6, activation='relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [120]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(name='accuracy')])

In [122]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("model/best_model",
                                            save_best_only=True)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(X_train,y_train,epochs=40,batch_size=512,
validation_data=(X_val, y_val),verbose=1,callbacks=[stop_early,checkpoint])

Epoch 1/40
274/274 [==============================] - ETA: 0s - loss: 0.3830 - accuracy: 0.7966INFO:tensorflow:Assets written to: model/best_model/assets


INFO:tensorflow:Assets written to: model/best_model/assets


274/274 [==============================] - 19s 69ms/step - loss: 0.3830 - accuracy: 0.7966 - val_loss: 0.1923 - val_accuracy: 0.9343
Epoch 2/40
274/274 [==============================] - ETA: 0s - loss: 0.2643 - accuracy: 0.8832INFO:tensorflow:Assets written to: model/best_model/assets


INFO:tensorflow:Assets written to: model/best_model/assets


274/274 [==============================] - 16s 60ms/step - loss: 0.2643 - accuracy: 0.8832 - val_loss: 0.1564 - val_accuracy: 0.9405
Epoch 3/40
273/274 [============================>.] - ETA: 0s - loss: 0.2081 - accuracy: 0.8992INFO:tensorflow:Assets written to: model/best_model/assets


INFO:tensorflow:Assets written to: model/best_model/assets


274/274 [==============================] - 17s 62ms/step - loss: 0.2081 - accuracy: 0.8992 - val_loss: 0.1490 - val_accuracy: 0.9428
Epoch 4/40
274/274 [==============================] - 14s 50ms/step - loss: 0.1874 - accuracy: 0.9053 - val_loss: 0.1501 - val_accuracy: 0.9423
Epoch 5/40
274/274 [==============================] - 14s 50ms/step - loss: 0.1724 - accuracy: 0.9092 - val_loss: 0.1559 - val_accuracy: 0.9414
Epoch 6/40
274/274 [==============================] - 14s 53ms/step - loss: 0.1624 - accuracy: 0.9123 - val_loss: 0.1619 - val_accuracy: 0.9406


In [123]:
loaded_model = tf.keras.models.load_model('model/best_model')
results = loaded_model.evaluate(X_test, y_test)

print(results)


32/32 [==============================] - 0s 5ms/step - loss: 0.1498 - accuracy: 0.9390
[0.14983966946601868, 0.9390000104904175]


In [124]:
y_hat = loaded_model.predict([X_test])

In [125]:
pred=[]
for i in y_hat:
  if i[0]<.5:
    i=0
  else:
    i=1
  pred.append(i)

In [126]:
from sklearn.metrics import classification_report,confusion_matrix

print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94       504
           1       0.94      0.94      0.94       496

    accuracy                           0.94      1000
   macro avg       0.94      0.94      0.94      1000
weighted avg       0.94      0.94      0.94      1000



In [127]:
my_text ="What do you call a bee that can’t make up its mind? A maybe."
y_hat = loaded_model.predict([my_text])
print(y_hat)
if y_hat > .5:
    print('Humor text')
else: 
    print('Non_Humor text')

[[0.9870832]]
Humor text


In [130]:
my_text2 ="We would love to hear your thoughts and feedback"
y_hat2 = loaded_model.predict([my_text2])
print(y_hat2)
if y_hat2 > .5:
    print('Humor text')
else: 
    print('Non_Humor text')

[[0.40188783]]
Non_Humor text
